In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from scipy.special import softmax
from sklearn.preprocessing import OneHotEncoder

In [ ]:
class ESN:
    def __init__(self, input_size, reservoir_size, output_size, spectral_radius=0.9, alpha=0.99, density=0.5):
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.spectral_radius = spectral_radius
        self.alpha = alpha

        # Input weight matrix (including bias)
        self.W_in = np.random.rand(reservoir_size, input_size + 1) - 0.5

        # Reservoir weight matrix
        self.W_res = np.zeros((reservoir_size, reservoir_size))
        #max_edges = reservoir_size ** 2  # Maximum possible edges (for a graph, in which a node can also be connected to itself), otherwise d_max = n(n-1)/2
        max_edges = reservoir_size * (reservoir_size - 1)
        num_edges = int(density * max_edges)  # Number of edges based on density
        indices = np.random.choice(max_edges, num_edges, replace=False) # Randomly assign `num_edges` connections
        self.W_res.flat[indices] = np.random.rand(num_edges) - 0.5  # Random weights between -0.5 and 0.5

        # Scale the reservoir weights to enforce spectral radius
        self.W_res *= spectral_radius / np.max(np.abs(np.linalg.eigvals(self.W_res)))

        # Output weight matrix
        self.W_out = np.random.rand(output_size, reservoir_size) - 0.5

        # Initial state
        self.x0 = np.random.rand(reservoir_size)

    def train(self, X_train, y_train, transient=100):
        X_train = np.concatenate((np.ones((len(X_train), 1)), X_train), axis=1)  # Add bias term to input
        X_res = np.zeros((len(X_train), self.reservoir_size))
        x = np.zeros(self.reservoir_size)

        for t in range(len(X_train)):
            u = X_train[t]
            x = self.alpha * np.tanh(np.dot(self.W_in, u) + np.dot(self.W_res, self.x0))
            if t > transient:
                X_res[t] = x

        self.W_out = np.dot(np.linalg.pinv(X_res[transient:]), y_train[transient:])

    def predict(self, X_test):
        X_test = np.concatenate((np.ones((len(X_test), 1)), X_test), axis=1)  # Add bias term to input
        X_res = np.zeros((len(X_test), self.reservoir_size))
        x = np.zeros(self.reservoir_size)

        for t in range(len(X_test)):
            u = X_test[t]
            x = self.alpha * np.tanh(np.dot(self.W_in, u) + np.dot(self.W_res, self.x0))
            X_res[t] = x

        return np.dot(X_res, self.W_out)

    def identity(self, x):
        return softmax(x)


In [ ]:
# Load MNIST data
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]
X = X / 255.0  # Normalize pixel values to range [0, 1]
X = np.array(X)
y = OneHotEncoder().fit_transform(y.values.reshape(-1, 1)).toarray()

print(X.shape, y.shape)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# MNIST classification using ESN with a network of 1000 neurons and alpha=1.0 and density=0.5
alpha = 1.0
reservoir_size = 1000
density = 0.5
output_size = 10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
input_size = X_train.shape[1]

esn = ESN(input_size, reservoir_size, output_size, alpha=alpha, density=density)
esn.train(X_train, y_train)

# Prediction
predictions = []
for i in X_test:
    predictions.append(esn.predict(i.reshape(1, -1)))

predictions = np.array(predictions).reshape(-1, 10)
accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
print("Accuracy:", accuracy)
